<a href="https://colab.research.google.com/github/rjrizani/Chrome-Extension/blob/master/synthetic_features_and_outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2017 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fitur Sintetis dan Pencilan

**Tujuan Pembelajaran:**
  * Membuat fitur sintetis yang merupakan rasio dari dua fitur lainnya
  * Menggunakan fitur baru ini sebagai masukan untuk model regresi linear
  * Meningkatkan efektivitas model dengan mengidentifikasi dan memotong (menghapus) pencilan dari data masukan

Mari kita lihat kembali model kita dari latihan Langkah Pertama yang lalu dengan TensorFlow.

Pertama, kita akan mengimpor data perumahan di California ke sebuah `DataFrame` *pandas*:

## Penyiapan

In [1]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

TensorFlow 1.x selected.


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
14263,-122.1,37.9,35.0,880.0,139.0,352.0,132.0,6.9,406.5
2681,-117.7,34.1,11.0,7392.0,1796.0,3841.0,1621.0,2.8,163.0
6095,-118.2,34.2,47.0,1853.0,345.0,757.0,310.0,3.7,422.0
15962,-122.4,37.7,35.0,2878.0,564.0,1633.0,528.0,4.5,266.9
10791,-120.7,35.3,16.0,164.0,30.0,542.0,32.0,1.7,42.5
...,...,...,...,...,...,...,...,...,...
321,-116.9,32.7,13.0,3268.0,491.0,1431.0,503.0,5.8,259.9
165,-116.2,33.7,17.0,4874.0,1349.0,5032.0,1243.0,2.4,90.0
5329,-118.2,34.2,37.0,1997.0,361.0,1037.0,363.0,3.8,210.3
2898,-117.8,34.0,13.0,984.0,127.0,364.0,119.0,7.6,426.9


Selanjutnya, kita akan menyiapkan fungsi masukan dan mendefinisikan fungsi untuk pelatihan model:

In [ ]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [ ]:
def train_model(learning_rate, steps, batch_size, input_feature):
  """Trains a linear regression model.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `california_housing_dataframe`
      to use as input feature.
      
  Returns:
    A Pandas `DataFrame` containing targets and the corresponding predictions done
    after training the model.
  """
  
  periods = 10
  steps_per_period = steps / periods

  my_feature = input_feature
  my_feature_data = california_housing_dataframe[[my_feature]].astype('float32')
  my_label = "median_house_value"
  targets = california_housing_dataframe[my_label].astype('float32')

  # Create input functions.
  training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)
  
  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column(my_feature)]
    
  # Create a linear regressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
  )

  # Set up to plot the state of our model's line each period.
  plt.figure(figsize=(15, 6))
  plt.subplot(1, 2, 1)
  plt.title("Learned Line by Period")
  plt.ylabel(my_label)
  plt.xlabel(my_feature)
  sample = california_housing_dataframe.sample(n=300)
  plt.scatter(sample[my_feature], sample[my_label])
  colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, periods)]

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  root_mean_squared_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period,
    )
    # Take a break and compute predictions.
    predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])
    
    # Compute loss.
    root_mean_squared_error = math.sqrt(
      metrics.mean_squared_error(predictions, targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    root_mean_squared_errors.append(root_mean_squared_error)
    # Finally, track the weights and biases over time.
    # Apply some math to ensure that the data and line are plotted neatly.
    y_extents = np.array([0, sample[my_label].max()])
    
    weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
    bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')
    
    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[period]) 
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Periods')
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(root_mean_squared_errors)

  # Create a table with calibration data.
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = pd.Series(predictions)
  calibration_data["targets"] = pd.Series(targets)
  display.display(calibration_data.describe())

  print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)
  
  return calibration_data

## Tugas 1: Cobalah Fitur Sintetis

Fitur `total_rooms` dan `population` menghitung jumlah total dari blok kota tertentu.

Namun, bagaimana jika satu blok kota memiliki tingkat kepadatan penduduk yang lebih tinggi daripada kota lainnya? Kita dapat melihat hubungan kepadatan blok dan median nilai rumah dengan membuat fitur sintetis yang merupakan rasio `total_rooms` dan `population`.

Dalam sel di bawah, buatlah fitur dengan nama `rooms_per_person`, lalu gunakan fitur tersebut sebagai `input_feature` untuk `train_model()`.

Apa performa terbaik yang bisa Anda dapatkan menggunakan satu fitur ini dengan mengubah kecepatan pembelajaran? (Semakin baik performanya, maka semakin baik pula kesesuaian garis regresi dengan data, dan semakin rendah pula nilai GARK-nya).

**CATATAN**: Anda mungkin terbantu dengan menambahkan beberapa sel di bawah sehingga Anda dapat mencoba beberapa kecepatan pembelajaran yang berbeda dan membandingkan hasilnya. Untuk menambahkan sel kode baru, arahkan kursor tepat di bawah pusat sel ini, lalu klik **KODE**.

In [ ]:
#
# YOUR CODE HERE
#
california_housing_dataframe["rooms_per_person"] =

calibration_data = train_model(
    learning_rate=0.00005,
    steps=500,
    batch_size=5,
    input_feature="rooms_per_person"
)

### Solusi

Klik di bawah untuk mendapatkan solusinya.

In [ ]:
california_housing_dataframe["rooms_per_person"] = (
    california_housing_dataframe["total_rooms"] / california_housing_dataframe["population"])

calibration_data = train_model(
    learning_rate=0.05,
    steps=500,
    batch_size=5,
    input_feature="rooms_per_person")

## Tugas 2: Identifikasi Pencilan

Kita dapat menggambarkan performa model dengan membuat bagan sebar dari prediksi vs. nilai target. Idealnya, prediksi dan nilai target akan berada pada garis diagonal yang berkorelasi dengan sempurna.

Gunakan [`scatter()`](https://matplotlib.org/gallery/shapes_and_collections/scatter.html) Pyplot untuk membuat bagan sebar dari prediksi vs. target, dengan menggunakan model rooms_per_person yang telah dilatih di Tugas 1.

Apakah Anda melihat ada kejanggalan? telusuri kejanggalan-kejanggalan kembali ke sumber data dengan melihat distribusi nilai di `rooms_per_person`.

In [ ]:
# YOUR CODE HERE

### Solusi

Klik di bawah untuk mendapatkan solusinya.

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.scatter(calibration_data["predictions"], calibration_data["targets"])

Data kalibrasi menunjukkan bahwa sebagian besar titik sebar sejajar dengan garis. Garis tersebut hampir vertikal, tetapi kita akan membahasnya lagi nanti. Untuk saat ini, mari fokus pada titik yang menyimpang dari garis. Dapat dilihat bahwa titik yang menyimpang jumlahnya relatif sedikit.

Jika menggambarkan histogram `rooms_per_person`, kita akan melihat bahwa ada beberapa pencilan dalam data masukan:

In [ ]:
plt.subplot(1, 2, 2)
_ = california_housing_dataframe["rooms_per_person"].hist()

## Tugas 3: Potong Pencilan

Lihat apakah kesesuaian model dapat ditingkatkan lebih lanjut dengan menyetel nilai pencilan dari `rooms_per_person` ke beberapa nilai maksimum atau minimum yang wajar.

Untuk referensi, berikut contoh singkat tentang cara menerapkan fungsi ke `Series` Pandas:

    clipped_feature = my_dataframe["my_feature_name"].apply(lambda x: max(x, 0))

`clipped_feature` di atas tidak akan memiliki nilai yang kurang dari `0`.

In [ ]:
# YOUR CODE HERE

### Solusi

Klik di bawah untuk mendapatkan solusinya.

Histogram yang kita buat di Tugas 2 menunjukkan bahwa sebagian besar nilainya kurang dari `5`. Mari kita potong nilai `rooms_per_person` ke angka 5, dan gambarkan histogram untuk memeriksa kembali hasilnya.

In [ ]:
california_housing_dataframe["rooms_per_person"] = (
    california_housing_dataframe["rooms_per_person"]).apply(lambda x: min(x, 5))

_ = california_housing_dataframe["rooms_per_person"].hist()

Untuk memverifikasi bahwa pemotongan berfungsi, mari kita latih dan cetak data kalibrasi sekali lagi:

In [ ]:
calibration_data = train_model(
    learning_rate=0.05,
    steps=500,
    batch_size=5,
    input_feature="rooms_per_person")

In [ ]:
_ = plt.scatter(calibration_data["predictions"], calibration_data["targets"])